# Projet  : Find in translation

##### chaque jour des dizaines d'objets sont perdus et retrouvés dans les gares parisiennes par les voyageurs, leur gestion est critique au niveau de la satisfaction client. Cependant le cout de leur gestion est critique également. On aimerait donc dimensionner au mieux le service en charge de les gérer mais pour cela il faut pouvoir anticiper de manière précise le volume d'objets trouvés chaque jour. Votre manager a une intuition qu'il aimerait vérifier: plus il fait froid plus les voyageurs sont chargés (manteau, écharppes, gant) plus ils ont donc de probabilité de les oublier. Mais empiler toutes ces couches prend du temps, ce qui pousse aussi à se mettre en retard et dans la précipitation, à oublier d'autres affaires encore. A l'aide des données de la SNCF et d'autres données, essayez de creuser cette piste.

#### 1) Requeter la base de données des objets trouvés pour récupérer les données entre 2019 et 2022 sur les gares parisiennes et Stocker les données dans une BDD SQL 

In [40]:
import requests
import pandas as pd
import sqlite3


#url = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=gc_obo_gare_origine_r_name+%3D+%22Paris%22+AND+(date%3D'2019'+OR+date%3D'2020'+OR+date+%3D'2021'+OR+date%3D'2022')&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&timezone=Europe%2FParis&rows=-1"

url = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=gc_obo_gare_origine_r_name+%3D+%22Paris%22+AND+(date%3D'2019'+OR+date%3D'2020'+OR+date+%3D'2021'+OR+date%3D'2022')&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&timezone=Europe%2FParis&rows=-1"

response = requests.get(url)
data = response.json()
facet = data["parameters"]["facet"]


#---------------------------------------------------------------------------------------------------------------
# Extraire les valeurs de facet
facets = facet
date = facets[0]
gc_obo_date_heure_restitution_c = facets[1]
gc_obo_gare_origine_r_name = facets[2]
gc_obo_nature_c = facets[3]
gc_obo_type_c = facets[4]
gc_obo_nom_recordtype_sc_c = facets[5]

# Extraire les champs des enregistrements
records = data["records"]
record_fields = []
for record in records:
    record_fields.append(record["fields"])

#print(record_fields)

# Créer un DataFrame à partir des champs des enregistrements
df = pd.DataFrame(record_fields)


# Écrire le DataFrame dans un fichier CSV
df.to_csv("objets-trouves-limite.csv", index=False)


# Créer une connexion à la base de données
connexion = sqlite3.connect('bdd.db')

df.to_sql('objets_trouves-limite', connexion,if_exists='replace', index=False)

connexion.close()


#### Requeter pour avoir une base de données complète

In [41]:
import requests
import pandas as pd
import sqlite3
import urllib.parse


url1 = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=objets-trouves-restitution&q=gc_obo_gare_origine_r_name+%3D+%22"
url2 = "&sort=date&facet=date&facet=gc_obo_date_heure_restitution_c&facet=gc_obo_gare_origine_r_name&facet=gc_obo_nature_c&facet=gc_obo_type_c&facet=gc_obo_nom_recordtype_sc_c&timezone=Europe%2FParis&rows=-1"


gares = ['Paris Gare de Lyon', 'Paris Montparnasse', 'Paris Gare du Nord', 'Paris Saint-Lazare', 'Paris Est', 'Paris Bercy', 'Paris Austerlitz']
date =[2019, 2020,2021,2022]

data_frames = []
for g in gares:
    for d in date:
        api_url = f"{url1}{urllib.parse.quote(g)}%22+AND+date%3D{str(d)}{url2}"
        response = requests.get(api_url)
        data = response.json()
        records = data["records"]
        record_fields = [record["fields"] for record in records]
        df = pd.DataFrame(record_fields)
        data_frames.append(df)

df_concatenated = pd.concat(data_frames, ignore_index=True)


# Créer un DataFrame à partir des champs des enregistrements
df = pd.DataFrame(df_concatenated)


# Écrire le DataFrame dans un fichier CSV
df.to_csv("objets-trouves.csv", index=False)

# Créer une connexion à la base de données
connexion = sqlite3.connect('bdd.db')

df.to_sql('objets_trouves', connexion,if_exists='replace', index=False)

connexion.close()


#### 2) Requeter la base de données de Logtitude et Latitude et Stocker les données dans une BDD SQL 

In [4]:
import requests

url_gares = "https://ressources.data.sncf.com/api/records/1.0/search/?dataset=referentiel-gares-voyageurs&q=&sort=gare_alias_libelle_noncontraint&rows=-1"


response = requests.get(url_gares)

data_list = []

if response.status_code == 200:
    data = response.json()
    for record in data['records']:
        gare_alias_libelle_noncontraint = record['fields']['gare_alias_libelle_noncontraint']
        if 'wgs_84' in record['fields']:
            latitude = record['fields']['wgs_84'][0]
            longitude = record['fields']['wgs_84'][1]
            data_list.append([gare_alias_libelle_noncontraint, latitude, longitude])

            
else:
    print("Une erreur s'est produite lors de la requête à l'API.")
    
df_data = pd.DataFrame(data_list, columns=[
    "Gare",
    "Latitude",
    "Longitude"
])

df_data.to_csv('df_data.csv')

import sqlite3
import pandas as pd 

connexion = sqlite3.connect('bdd.db')

df_data.to_sql('coordonnees_gares', connexion,if_exists='replace', index=False)

connexion.close()


#### 2) Requeter la base de données de la température et Stocker les données dans une BDD SQL 